### Note: the NewsQA I found did not have any unanswerable questions in the dataset; which is why I moved forward with Squadv2

# Data Loading and Imports

In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from typing import List, Dict

squad_v2 = load_dataset("squad_v2", split="train[:90%]")
squad_v2 = squad_v2.train_test_split(test_size=0.1)

train = squad_v2["train"]
test = squad_v2["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [2]:
df = train.to_pandas().sample(n = 10000).reset_index(drop= True)
df.head()

,id,title,context,question,answers
0,573009aca23a5019007fcc8d,San_Diego,San Diego County has one of the highest counts...,How many bird species have been observed in Sa...,"{'text': ['492'], 'answer_start': [277]}"
1,5ad3abec604f3c001a3feb97,High-definition_television,"In 1979, the Japanese state broadcaster NHK fi...",In what year did NHK first develop consumer SD...,"{'text': [], 'answer_start': []}"
2,56f9f848f34c681400b0bf0f,Somalis,"Somalis (Somali: Soomaali, Arabic: صومال‎) are...",What country has the second largest Somali pop...,"{'text': ['Ethiopia'], 'answer_start': [389]}"
3,5727be6f3acd2414000deb3b,Tibet,The modern Standard Chinese exonym for the eth...,During which dynasty was the term Xizang first...,"{'text': ['Qing'], 'answer_start': [509]}"
4,570a01b44103511400d594ec,Houston,"By 1860, Houston had emerged as a commercial a...",What commodity was Houston known for in 1860?,"{'text': ['cotton'], 'answer_start': [80]}"


In [3]:
len(df)

10000

#### check for null rows

In [4]:
df.isnull().sum()

,0
id,0
title,0
context,0
question,0
answers,0


### Check for number of no-direct answer from the context.

In [5]:
in_context = []
for i in range(len(df)):
    curr =df['answers'][i]
    if len(curr["text"]) == 0:
        in_context.append(1)
        continue
    index = df['context'][i].find(curr["text"][0])
    if index != -1:
        in_context.append(0)
sum(in_context)

3349

#### Check for number of rows with more than one answer for a single context and question

In [6]:
more_than_one_ans = []
for i in range(len(df)):
    curr =df['answers'][i]
    if len(curr["answer_start"])>1:
        more_than_one_ans.append(1)
    else:
        more_than_one_ans.append(0)
sum(more_than_one_ans)


0

### Update the "answers" column to hold end index of the answer

In [7]:
labels = []
num_correct = 0;           # to recheck if the substring from context is same as answer.
for i in range(len(df)):
  curr = df["answers"][i]
  if len(curr["answer_start"]) > 0:
    curr["answer_end"] = [curr["answer_start"][0] + len(curr["text"][0])]
    answer_start = curr["answer_start"][0]
    answer_end = curr["answer_end"][0]
    if df["context"][i][answer_start:answer_end] == curr["text"][0]:
      num_correct+=1      # only calculated for case when answer was present. we can confirm that the indices are correct as 6651(below output)+3349(unanswerable) = 10000
  else:
    curr["answer_end"] =[]
  labels.append(curr)

df["answers"] = labels
print(num_correct)

6651


## Import roBERTa base to fine tune on our dataset

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers.trainer_utils import EvalPrediction

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model.config.hidden_dropout_prob = 0.2

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model.config

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [10]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_data = dataset['train']
val_data = dataset['test']

## Data Preprocessing:


*   Tokenization
*   Calculation of Answer span in the TOKENIZED context with the help of provided answer span in the NON TOKENIZED context; this is done as the model only takes token indices and not character positions
*   The offset mapping maps each token in the tokenized sequence to its corresponding character positions in the original, non-tokenized text



In [ ]:
def preprocess(dataset):
    questions = [question.strip() for question in dataset['question']]       # cleaning the data
    contexts = [context.strip() for context in dataset['context']]

    inputs = tokenizer(                                 # tokinizing
        questions,
        contexts,
        max_length=384,                                 # could have kept 512, but took more time so asked chatgpt a balanced length- gave 384 as it was used for original squad dataset
        truncation= "only_second",
        return_offsets_mapping = True,
        padding = "max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")       # to process separately

    answers = dataset['answers']
    start_positions = []                                # to store start and end pos in the *tokenized* sequence
    end_positions = []

    for i, ans_list in enumerate(answers):
        if len(ans_list) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Get first answer span
        ans = ans_list

        if len(ans["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = ans["answer_start"][0]     # this extra 0 index is present as ans["answer_start/end"] are arrays of single elements (or no elements- handled above)
        end_char = ans["answer_end"][0]

        seq_ids = inputs.sequence_ids(i)        # Map char positions to token indices using offsets
        # to find Start position of the token:
        idx = 0
        while idx<len(offset_mapping[i]) and offset_mapping[i][idx][0] < start_char:  
            idx+=1

        if idx<len(seq_ids)-1 and seq_ids[idx]==1:
            start_pos = idx
        else:
            start_pos = 0
        # to find end position of the token:
        idx = start_pos
        while idx<len(offset_mapping[i]) and offset_mapping[i][idx][1] <= end_char:
            idx+=1

        if idx <= len(seq_ids) and seq_ids[idx-1]==1:
            end_pos = idx - 1
        else:
            end_pos = 0

        context_start = 0
        while context_start < len(seq_ids) and seq_ids[context_start] != 1:  #It ensures start_pos is not before the context start
            context_start += 1
        context_end = len(seq_ids) - 1

        while context_end >= 0 and seq_ids[context_end] != 1:                # Ensures end_pos is not after the context end, preventing answers from being incorrectly placed in the question or padding tokens.
            context_end -= 1
        start_pos = max(start_pos, context_start)
        end_pos = min(end_pos, context_end)

        if start_pos>end_pos:
            start_pos = 0
            end_pos = 0

        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [12]:
tokenized_train = train_data.map(preprocess, batched = True, remove_columns = train_data.column_names)
tokenized_val = val_data.map(preprocess, batched = True, remove_columns = val_data.column_names)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Training

In [ ]:
from transformers.trainer_utils import EvalPrediction
from evaluate import load 

In [ ]:
def compute_metrics(eval_pred):
    predictions, _ = eval_pred.predictions, eval_pred._
    pred_start_logits, pred_end_logits = predictions
    start_probs = torch.softmax(torch.tensor(pred_start_logits), dim=-1)[:, 0].numpy()   
    end_probs = torch.softmax(torch.tensor(pred_end_logits), dim=-1)[:, 0].numpy()
    no_answer_probs = (start_probs + end_probs) / 2                                     

    pred_start = np.argmax(pred_start_logits, axis=-1)          # take max probable start and end positions 
    pred_end = np.argmax(pred_end_logits, axis=-1)

    formatted_preds = []
    for i in range(len(pred_start)):
        if no_answer_probs[i] > 0.5 or pred_start[i] == 0 or pred_end[i] == 0 or pred_start[i] > pred_end[i]:
            formatted_preds.append({"id": str(i), "prediction_text": "", "no_answer_probability": float(no_answer_probs[i])})
        else:
            if pred_start[i] <= pred_end[i] < len(tokenized_val[i]["input_ids"]):
                pred_text = tokenizer.decode(tokenized_val[i]["input_ids"][pred_start[i]:pred_end[i] + 1], skip_special_tokens=True)
            else:
                pred_text = ""
            formatted_preds.append({
                "id": str(i),
                "prediction_text": pred_text.strip(),
                "no_answer_probability": float(no_answer_probs[i])
            })

    formatted_labels = []
    for i in range(len(val_data)):
        ans_dict = val_data[i]["answers"]
        if len(ans_dict["text"]) == 0 or ans_dict["text"][0] == "":
            formatted_labels.append({"id": str(i), "answers": {"text": [""], "answer_start": [0]}})
        else:
            text_array = ans_dict["text"]
            start_pos_array = ans_dict["answer_start"]
            formatted_labels.append({"id": str(i), "answers": {"text": text_array, "answer_start": start_pos_array}})

    metric = load("squad_v2")
    results = metric.compute(predictions=formatted_preds, references=formatted_labels)

    return {
        "exact_match": results["exact"],
        "f1": results["f1"],
    }

In [16]:
training_args = TrainingArguments(
    output_dir = './qa-fine_tune',
    eval_strategy = 'epoch',
    learning_rate = 3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    warmup_steps=500,
    save_strategy = "steps",
    dataloader_num_workers=4,
    metric_for_best_model="eval_f1",
    fp16 = True,
)

In [17]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer = tokenizer,
    compute_metrics= compute_metrics
)
trainer.train()

/tmp/ipython-input-1992543726.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vivekkashyap-kashyap124 (vivekkashyap-kashyap124-nitk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Exact Match,F1
1,2.907200,1.420862,49.500000,55.029310
2,1.260600,1.199851,56.100000,61.772222
3,0.877600,1.224025,58.200000,64.700909
4,0.579000,1.394772,57.600000,65.775470
5,0.408700,1.582669,58.500000,66.622271


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: T

TrainOutput(global_step=2815, training_loss=1.1053515527430477, metrics={'train_runtime': 1097.8352, 'train_samples_per_second': 40.99, 'train_steps_per_second': 2.564, 'total_flos': 8818765539840000.0, 'train_loss': 1.1053515527430477, 'epoch': 5.0})

## Example

In [1]:
from transformers import pipeline
import torch

model_checkpoint = r"D:\Personal\code\ML comp\New folder\checkpoint-2815"  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

qa_pipeline = pipeline(
    "question-answering",
    model=model_checkpoint,
    tokenizer=model_checkpoint,
    device=device
)

def extractive_qa(sample_question, sample_context):
    result = qa_pipeline(question=sample_question, context=sample_context)
    if result['score']<0.1:
        print("Could not find the answer in the context")
        return
    print("Sample Prediction:")
    print(f"Question: {sample_question}")
    print(f"Context: {sample_context}")
    print(f"Answer: '{result['answer']}' (score: {result['score']:.3f}, start: {result['start']}, end: {result['end']})")

d:\Personal\code\ML comp\New folder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda


In [2]:
sample_question = "Which team represented the AFC at Super Bowl 50"  
sample_context = "Super Bowl 50 was an American football game... The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10"
extractive_qa(sample_question, sample_context)

Sample Prediction:
Question: Which team represented the AFC at Super Bowl 50
Context: Super Bowl 50 was an American football game... The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10
Answer: 'Denver Broncos' (score: 0.968, start: 95, end: 109)


In [3]:
sample_question = "Which team represented the KFC at Burger King 50"  
sample_context = "Super Bowl 50 was an American football game... The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10"
extractive_qa(sample_question, sample_context)

Could not find the answer in the context


#### Did not have time to think of another way to handle the unanswerable question rather than bruteforcing to find a threshold. but works good for most of the questions